In [ ]:

import sys
sys.path.append('..')
import json 
import os 
import pandas as pd 
import numpy as np 
import analysis_utils
import importlib 
from matplotlib.pylab import plt 
import importlib
import analysis_utils
from matplotlib.pylab import plt 
import seaborn as sns 
import numpy as np
import os
import json
import pandas as pd
import analysis_utils
import random
from scipy.stats import pearsonr, spearmanr
from scipy.stats import wasserstein_distance_nd

import marshaling.process_model as model_utils

%matplotlib inline

In [ ]:
''' 
Load human data
'''

In [ ]:

''' 
NOTE --- this may need to change
Load in data, per query, per condition
'''



''' 
Key other function from analysis_utils to change
'''


def load_and_process_samples_from_inference_results(inference_results_path, inference_file, n_queries):
    with open(f"{inference_results_path}{inference_file}", "r") as f: 
        inf_data = json.load(f)
        rollout_idx = int(inference_file.split("inference_results_")[-1].split(".json")[0])
        executability = inf_data['wm']['executability']
        if executability == -1: return 

        # MCMC samples 
        # TODO ----- no longer 'model_metadata'
        #full_samples = inf_data['wm']['model_metadata']['official_posterior_samples']
        full_samples = inf_data['wm']['official_posterior_samples']   
        # NEW
        intermediate_samples = inf_data['wm']['intermediate_posterior_samples']   
        
        
        # TODO ----- add option to load in per query ---- add in "samples to process"
        # CHECK that this has the samples for the ablated versions?
        
        # Full synthesis model. This could include the ablated commentary variants later on.
        samples_to_process = {"full" : full_samples}#, "intermediate": intermediate_samples}
        # per condition
        for condition_idx, cond_samples in enumerate(intermediate_samples): 
            samples_to_process[f'cond_{condition_idx}'] = cond_samples

        processed_samples = {}
        for samp_name, samples in samples_to_process.items():
            if samples is None or len(samples) == 0: continue
            if type(samples) == str: samples = eval(samples)
            
            # minimal processing of samples into query format
            # {query1: [samples], query2: [samples]}
            query_tags = set(samples[0]["value"].keys()) # all queries are computed per sample 
            if len(query_tags) != n_queries: 
                print("MISSING QUERIES: ", query_tags)
                continue
            
            query_samples = {query: [] for query in query_tags}
            for entry in samples: 
                for query, val in  entry["value"].items():
                    query_samples[query].append(val)
                    
            # process the values to make sure they are b/w 0-100
            query_samples_processed = {query: [] for query in query_samples}
            for query, values_list in query_samples.items(): 
                query_idx = int(query.split("query")[-1])
                if query_idx <= 8: 
                    if type(values_list[0]) == bool:
                        print('wrong response form: ', query_idx, values_list[0])
                        assert False
                    elif np.max(values_list) <= 1: 
                        values_list = [v*100 for v in values_list]

                query_samples_processed[query] = values_list    
            processed_samples[samp_name] = query_samples_processed
    return processed_samples




importlib.reload(analysis_utils)

processed_vals_all_srcs =analysis_utils.process_values_from_all_sources(
    all_scenarios=all_scenarios,
    llm_res=llm_res,
    baseline_human_data=baseline_human_data,
    all_inference_data=all_inference_data,
    gold_inference_data=all_gold_data,
    n_queries=n_queries,
    method="flat"
)
